In [2]:
import os
from PIL import Image
import pytesseract
import re
from pdf2image import convert_from_path

import logging
import sys

import pandas as pd

In [102]:
arvd_strs = "|".join(['Arvd~', 'Arvd-'])
clrd_strs = "|".join(['Clrd-', 'Cird-'])

def find_between(entry_text, left_text, right_text):
    left_end = re.search(left_text, entry_text)
    right_start = re.search(right_text, entry_text)
    
    if left_end and right_start:
        return entry_text[left_end.end():right_start.start()].strip()
    else:
        return None
    

In [125]:

        
def parse_entry(entry_text):
    entry_words = [w for w in entry_text.split(' ') if len(w) > 0]

    # call number is always the first word of the entry
    call_number = entry_words[0]

    # the next 'word' is always the time
    call_time = entry_words[1][:4]
    
    # call reason
    call_reason =  find_between(entry_text, call_time, 'Call Taker')
    
    # call taker is always the string between taker and location
    call_taker =  find_between(entry_text, 'Call Taker', 'Location/Addr')
    
    return [call_number, call_time, call_reason, call_taker]


def replace_none_with_value(left_list, right_list):
    return [l if not l is None else r for l, r in zip(left_list, right_list)]


In [126]:
year_str = '19-'
current_date = '01/01/2019'

Arvd_list = []
Clrd_list = []


i_entry = 0
parsed_pages = []

for ipage in range(1,4):
    with open('../../data/Logs2019/page_{}.txt'.format(ipage),'r') as infile:
        page_text = infile.read()
    
    #print(ipage)
    #if ipage==2:
    #    print(page_text)
    
    
    page_entries = [log_idx.start() for log_idx in re.finditer(year_str, page_text)] + [-1]

    # check for updated date - we assume only once ever on the page
    date_end = [date_idx.end() for date_idx in re.finditer('For Date: ', page_text)]
    if len(date_end) > 1:
        print("Oh no, multiple days on the same page... this wont work.")
        adgasdgs
    
    # we found only one date so update
    elif len(date_end) == 1:
        current_date = re.search('([^\s]+)', page_text[date_end[0]:])
        if current_date:
            current_date = current_date.group(0).replace('-', "").strip()


    # now we worry about entries that start from the previous page
    initial_text = page_text[0:page_entries[0]]
    if len(initial_text) > 0 and len(parsed_pages) > 0:
        str_entry = parse_entry(initial_text)
        parsed_pages[-1][1:] = replace_none_with_value(parsed_pages[-1][1:], str_entry)
        
    
    for i_start in range(len(page_entries)-1):

        entry_text= page_text[page_entries[i_start]:page_entries[i_start + 1]]
        
        str_entry = parse_entry(entry_text)
        if str_entry[0] == '19-12':
            print(entry_text)
            print(str_entry)
        parsed_pages.append([current_date] + str_entry)
        i_entry += 1
        
        # worry about the arvd and cleared times
        if False and ('Arvd-' and 'Clrd-' in entry_text):
            # go into the end of column mode
            
            Arvd_list.extend([re.search('([^\s]+)', entry_text[arvd.end():]).group(0) for arvd in re.finditer(arvd_strs, entry_text)])
            
            Clrd_list.extend([re.search('([^\s]+)', entry_text[clrd.end():]).group(0)[:8] for clrd in re.finditer(clrd_strs, entry_text)])
        

19-12  1004  Initiated - ROAD CONDITIONS  SERVICES RENDERED  Call Taker:  PATROL DAVID JENNINGS, D  SOUTHWORTH ST  Location/Address:  Unit:  34  Arvd-10:05:59  Cird-10:07:54 
['19-12', '1004', 'Initiated - ROAD CONDITIONS  SERVICES RENDERED', ':  PATROL DAVID JENNINGS, D  SOUTHWORTH ST']


In [127]:
parsed_pages = pd.DataFrame(parsed_pages, columns = ['current_date', 'call_number', 'call_time', 'call_reason_action', 'call_taker'])                
parsed_pages


,current_date,call_number,call_time,call_reason_action,call_taker
0,01/01/2019,19-1,0341,Initiated - PARKING CHECK SERVICES RENDERED,: PATROL CRAIG A EICHHAMMER
1,01/01/2019,19-4,0834,Initiated - BUILDING CHECK BUILDING CHECKED/S...,": PATROL DAVID JENNINGS, D"
2,01/01/2019,19-5,0842,Initiated - BUILDING CHECK BUILDING CHECKED/S...,": PATROL DAVID JENNINGS, D"
3,01/01/2019,19-6,0846,Initiated - BUILDING CHECK BUILDING CHECKED/S...,": PATROL DAVID JENNINGS, D"
4,01/01/2019,19-7,0847,Initiated - BUILDING CHECK BUILDING CHECKED/S...,"PATROL DAVID JENNINGS, D"
5,01/01/2019,19-8,0859,Initiated ~ BUILDING CHECK BUILDING CHECKED/S...,"PATROL DAVID JENNINGS, D"
6,01/01/2019,19-9,0903,Initiated - BUILDING CHECK BUILDING CHECKED/S...,": PATROL DAVID JENNINGS, D"
7,01/01/2019,19-10,0906,Initiated - BUILDING CHECK BUILDING CHECKED/S...,": PATROL DAVID JENNINGS, D"
8,01/01/2019,19-11,0941,Initiated ~ BUILDING CHECK BUILDING CHECKED/S...,": PATROL DAVID JENNINGS, D"
9,01/01/2019,19-12,1004,Initiated - ROAD CONDITIONS SERVICES RENDERED,": PATROL DAVID JENNINGS, D SOUTHWORTH ST"
